In [1]:
import os
import torch
import numpy as np
import pandas as pd
import datetime as dt
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [3]:
PU_dt = 'tpep_pickup_datetime'
PU_loc = 'PULocationID'
PU_dow = 'day_of_week'
bucket = 't_bucket'
PU_dow_1 = 'day_of_week_oneHot'
bucket_1 = 't_bucket_oneHot'
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
csv_file = pd.read_csv(os.path.join('../assets', 'test.csv'),usecols=np.arange(17),parse_dates=[PU_dt],date_parser=dateparse)
# print(csv_file.keys())
# print(csv_file)

In [4]:
# loop to create day of week dictionary
day_vec = dict()
for i in range(7):
    oneHotVec = np.zeros((1,7)).astype('int')
    oneHotVec[0,i] = 1
    day_vec[i] = oneHotVec
            
# loop to create bucket dictionary
time_buckets = dict()
time_buckets_1 = dict()
split = 30
numBuckets = int(24*(60/split))
ind = 0
for hr in range(24):
    for i in range(int(60/split)):
        key = '%02d'%hr+':'+'%02d'%(i*split)+':00'
        oneHotVec = np.zeros((1,numBuckets)).astype('int')
        oneHotVec[0,ind] = 1
        time_buckets[key] = [ind, oneHotVec]
        ind += 1

In [5]:
# TODO: Fetch Convert date and time to day of week and time buckets of 30 mins.
# TODO: Fetch Pickup location ID
# TODO: Insert into numpy matrix


csv_file[PU_dt] = pd.to_datetime(csv_file[PU_dt])
csv_file[PU_dt] = csv_file[PU_dt].dt.round('30min')
csv_file[PU_dow] = csv_file[PU_dt].dt.dayofweek
csv_file[bucket] = csv_file[PU_dt].dt.time.apply(lambda x: time_buckets[str(x)][0])
csv_file[PU_dow_1] = (csv_file[PU_dt].dt.dayofweek).apply(lambda x: day_vec[x])
csv_file[bucket_1] = csv_file[PU_dt].dt.time.apply(lambda x: time_buckets[str(x)][1])

new_df = csv_file.groupby([PU_loc,PU_dow,bucket]).size().reset_index()
# print(new_df)


In [10]:
class taxiDataset(Dataset):
    """Taxi Data dataset."""
    
    def __init__(self, csv_file, root_dir, desired_labels, split):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
        """
        PU_dt = 'tpep_pickup_datetime'
        dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        df_all = pd.read_csv(os.path.join('../assets', csv_file),usecols=np.arange(17),parse_dates=[PU_dt],date_parser=dateparse)
        self.dayDict = self._createDayDict()
        self.timeBuckets = self._createTimeBuckets()
        self.df = self._parsetime(df_all, desired_labels, PU_dt, split)
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        PU_loc = self.df.iloc[idx, 0]
        PU_dow = self.dayDict[self.df.iloc[idx, 1]]
        PU_tB = self.timeBuckets[self.df.iloc[idx, 2]]
        PU_num = self.df.iloc[idx, 3]
        sample = {'PU_loc': PU_loc, 'PU_dow': PU_dow, 'PU_tB': PU_tB, 'PU_num': PU_num}

        return sample
    
    def _createDayDict(self):
        # loop to create day of week dictionary
        day_dict = dict()
        for i in range(7):
            oneHotVec = np.zeros((1,7)).astype('int')
            oneHotVec[0,i] = 1
            day_dict[i] = oneHotVec
        
        return day_dict
    
    def _createTimeBuckets(self):
        # loop to create time bucket dictionary
        time_buckets = dict()
        split = 30
        numBuckets = int(24*(60/split))
        ind = 0
        for hr in range(24):
            for i in range(int(60/split)):
                key = '%02d'%hr+':'+'%02d'%(i*split)+':00'
                oneHotVec = np.zeros((1,numBuckets)).astype('int')
                oneHotVec[0,ind] = 1
                time_buckets[key] = ind
                time_buckets[ind] = oneHotVec
                ind += 1
                
        return time_buckets
    
    def _parsetime(self, data_frame, desired_labels, dt_label, split):
        data_frame[dt_label] = pd.to_datetime(data_frame[dt_label])
        data_frame[dt_label] = data_frame[dt_label].dt.round('30min')
        data_frame[desired_labels[1]] = (data_frame[dt_label].dt.dayofweek)#.apply(lambda x: self.dayDict[x])
        data_frame[desired_labels[2]] = data_frame[dt_label].dt.time.apply(lambda x: self.timeBuckets[str(x)])
        
        return data_frame.groupby(desired_labels).size().reset_index()
        
        
        



In [11]:
PU_dt = 'tpep_pickup_datetime'
PU_loc = 'PULocationID'
PU_dow = 'day_of_week'
bucket = 't_bucket'
dl = [PU_loc,PU_dow,bucket]
taxi_dataset = taxiDataset(csv_file='test.csv', root_dir='test.csv', desired_labels=dl, split=30)
sample = taxi_dataset[3]

dataloader = DataLoader(taxi_dataset,batch_size=4,shuffle=True,num_workers=4)






